In [1]:
import warnings
from math import comb

import numpy as np
import pandas as pd
from scipy.stats import bernoulli, logistic

warnings.filterwarnings("ignore")

In [2]:
# Pooling====
# GROUP TESTING
# CREATED: Eli P. Fenichel, Yale University
# LAST UPDATED: July 15, 2020
# Probability of a groups testing positive accounting for sensitivity of test
# with prevalence m, group size = group, population = pop


def estimate_pooling_prob_of_pos_groups(group_size, m, pop, se, s_loss):
    shat = se - group_size * s_loss
    p_pos = 1 - (1 - shat * m) ** group_size  # shat es la sensibilidad,
    # p.pos: Probabilidad que un grupo sea positivo
    # (1-prevalencia): Probabilidad que se salga negativo una persona
    # (1-prevalencia)^tamano_grupo: Probabilidad que todos sean negativos en el grupo
    return p_pos

In [3]:
# Group results
def estimate_pooling_tests(group_size, p, pop, se, s_loss, sp):
    p_pos_pool = estimate_pooling_prob_of_pos_groups(group_size, p, pop, se, s_loss)
    p_pos_test = se * p + (1 - sp) * (1 - p)
    # number of groups
    n_group = int(np.round(pop / group_size))
    pos_groups = np.sum(bernoulli.rvs(p_pos_pool, size=n_group))
    neg_groups = n_group - pos_groups  # cambio

    # number of people
    neg_peop_group = neg_groups * group_size
    pos_peop_group = pos_groups * group_size

    pos_peop = pos_peop_group * p_pos_test

    n_groups_pooling = pos_groups + neg_groups
    total_tests_pooling = n_groups_pooling + pos_peop
    test_per_person_pooling = total_tests_pooling / pop

    # the expected number of people in negative groups that are actually
    # false positives with group size = group.size, prevalence p, sensitivity s,
    # in a population p.

    shat = se - group_size * s_loss
    t_prob = (1 - shat) * p + (1 - p)
    expected_number_missed = np.zeros(group_size - 1)
    for x in range(group_size - 1):
        expected_number_missed[x] = (
            comb(group_size, group_size - (x + 1))
            * ((1 - p) / t_prob) ** (group_size - (x + 1))
            * ((1 - shat) * p / t_prob) ** (x + 1)
        )

    exp_m = np.matmul(range(1, group_size), expected_number_missed)
    gm = exp_m * neg_groups

    res = pd.DataFrame(
        {
            "p_pos_pool": [p_pos_pool],
            "p_pos_test": [p_pos_test],
            "pos_groups": [pos_groups],
            "neg_groups": [neg_groups],
            "pos_peop_group": [pos_peop_group],
            "pos_peop": [pos_peop],
            "neg_peop_group": [neg_peop_group],
            "n_groups_pooling": [n_groups_pooling],
            "n_undetected_pos": [gm],
            "undetected_groups": [exp_m],
            "total_tests_pooling": [total_tests_pooling],
            "number_test_per_person_pooling": [test_per_person_pooling],
        }
    )
    return res

In [4]:
def covid_testing_strategies(
    prevalence,
    sens_model,
    spec_model,
    sens_ag_test,
    spec_ag_test,
    sens_pcr_test,
    spec_pcr_test,
    sens_lamp_test,
    spec_lamp_test,
    people,
    cost_pcr,
    cost_antigen,
    cost_lamp,
    group_size,
    strategy,
    budget,
):
    df_out = pd.DataFrame({})
    sens = sens_model
    spec = spec_model
    # for sens in sens_model:
    #     for spec in spec_model:

    # Probability that an a test gives a positive result
    # Antigen
    prob_res_pos_ag = sens_ag_test * prevalence + (1 - spec_ag_test) * (1 - prevalence)
    # LAMP
    prob_res_pos_lamp = sens_lamp_test * prevalence + (1 - spec_lamp_test) * (
        1 - prevalence
    )
    # PCR
    prob_res_pos_pcr = sens_pcr_test * prevalence + (1 - spec_pcr_test) * (
        1 - prevalence
    )
    # Probability that a test gives a negative result
    prob_res_neg_ag = 1 - prob_res_pos_ag
    prob_res_neg_lamp = 1 - prob_res_pos_lamp
    prob_res_neg_pcr = 1 - prob_res_pos_pcr

    # Probability model gives a low risk classification
    # prob_model_low_risk = (1 - sens) * prevalence +
    #   spec * (1 - prevalence)
    # prob_model_high_risk = 1 - prob_model_low_risk

    prob_model_high_risk_ag = sens * prob_res_pos_ag + (1 - spec) * prob_res_neg_ag
    # prob_model_high_risk_ag = 1 - prob_model_low_risk_ag

    prob_model_high_risk_pcr = sens * prob_res_pos_pcr + (1 - spec) * prob_res_neg_pcr
    # prob_model_high_risk_pcr = 1 - prob_model_low_risk_pcr

    q_model_high_risk_ag = logistic.ppf(prob_model_high_risk_ag)
    q_model_high_risk_pcr = logistic.ppf(prob_model_high_risk_pcr)

    prob_model_high_risk = logistic.cdf(
        0.5 * (q_model_high_risk_ag + q_model_high_risk_pcr)
    )
    prob_model_low_risk = 1 - prob_model_high_risk

    # PPV y NPV of model
    npv_model_ag = spec * prob_res_neg_ag / prob_model_low_risk
    npv_model_pcr = spec * prob_res_neg_pcr / prob_model_low_risk

    q_npv_model_low_risk_ag = logistic.ppf(npv_model_ag)
    q_npv_model_low_risk_pcr = logistic.ppf(npv_model_pcr)

    npv_model = logistic.cdf(0.5 * (q_npv_model_low_risk_ag + q_npv_model_low_risk_pcr))
    #
    # npv_model <- spec * (1 - prevalence) /
    #   ((1 - sens) * prevalence + spec * (1 - prevalence))

    # People in risk
    people_high_risk = np.sum(bernoulli.rvs(prob_model_high_risk, size=people))
    people_low_risk = people - people_high_risk

    # Probability that given High risk classification
    # the patient has a positive result with antigen

    prob_res_pos_given_high_risk_ag = sens * prob_res_pos_ag / prob_model_high_risk

    prob_res_neg_given_high_risk_ag = (
        (1 - spec) * prob_res_neg_ag / prob_model_high_risk
    )

    prob_res_pos_given_low_risk_ag = (1 - sens) * prob_res_pos_ag / prob_model_low_risk

    prob_res_pos_given_high_risk_lamp = sens * prob_res_pos_lamp / prob_model_high_risk

    prob_res_neg_given_high_risk_lamp = 1 - prob_res_pos_given_high_risk_lamp

    prob_res_pos_given_high_risk_pcr = sens * prob_res_pos_pcr / prob_model_high_risk

    prob_res_neg_given_high_risk_pcr = 1 - prob_res_pos_given_high_risk_pcr

    # Probability that given High risk classification
    # the patient has a negative result with antigen
    prob_res_neg_given_high_risk_ag = (
        (1 - spec) * prob_res_neg_ag / prob_model_high_risk
    )

    # PPV golden test
    ppv_golden_test = (
        sens_pcr_test
        * prevalence
        / (sens_pcr_test * prevalence + (1 - spec_pcr_test * (1 - prevalence)))
    )

    # Strategy 1 -------------------------------------------------------
    if strategy == 1:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            # Probability of people tested before of 5 days since symptoms onset.
            prob_symp_less_5d = 1 - prob_symp_more_5d

            #             people_low_risk = 0

            #             # Number of antigen tests done
            #             number_alt_tests = np.sum(
            #                 bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            #             )

            #             # Number of PCR tests done
            #             number_pcr_tests = (
            #                 people_high_risk
            #                 - number_alt_tests
            #                 + np.sum(
            #                     bernoulli.rvs(
            #                         prob_res_neg_given_high_risk_ag, size=number_alt_tests
            #                     )
            #                 )
            #             )

            #             number_test_per_person = (number_pcr_tests + number_alt_tests) / (
            #                 people_high_risk
            #             )

            #             number_positive_reported = (
            #                 people_res_pos_given_high_risk_ag
            #                 + people_res_pos_given_res_neg_ag_high_risk
            #                 + people_res_pos_given_high_risk_pcr
            #                 + pooling_tests_low_risk["pos_peop"]
            #             )

            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
            )

            number_alt_tests = people_symp_less_5d

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag, size=people_symp_less_5d)
            )
            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            number_pcr_tests = (
                people_high_risk
                - number_alt_tests
                + np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_ag, size=number_alt_tests
                    )
                )
            )
            number_test_per_person = (number_pcr_tests + number_alt_tests) / (
                people_high_risk
            )
            # Total costs.
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
            )

            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": [total_cost_alt],
                    "total_cost_pcr": [total_cost_pcr],
                    "total_cost": [total_cost],
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 2:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d

            pooling_tests_full_prevalence = pd.DataFrame()

            pooling_tests_full_prevalence = estimate_pooling_tests(
                group_size=group_size,
                p=prevalence,
                pop=people_low_risk,
                se=sens_pcr_test,
                s_loss=0,
                sp=spec_pcr_test,
            )

            pooling_tests_low_risk = estimate_pooling_tests(
                group_size=group_size,
                p=1 - npv_model,
                pop=people_low_risk,
                se=sens_pcr_test,
                s_loss=0,
                sp=spec_pcr_test,
            )

            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
            )

            number_alt_tests = people_symp_less_5d

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag, size=people_symp_less_5d)
            )
            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            number_pcr_tests = (
                people_high_risk
                - number_alt_tests
                + np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_ag, size=number_alt_tests
                    )
                )
                + pooling_tests_low_risk["total_tests_pooling"].values
            )
            number_test_per_person = (number_pcr_tests + number_alt_tests) / (people)
            # Total costs.
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
                + pooling_tests_low_risk["pos_peop"]
            )

            # number_positive_reported <-
            #   people_high_risk * prob_symp_less_5d *
            #   (
            #     prob_res_pos_given_high_risk_ag +
            #       prob_res_neg_given_high_risk_ag *
            #         prob_res_pos_given_high_risk_pcr
            #   ) +
            #   people_high_risk * prob_symp_more_5d *
            #     prob_res_pos_given_high_risk_pcr +
            #   pooling_tests_low_risk$pos_peop
            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 3:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d
            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
            )

            number_alt_tests = (
                people_symp_less_5d + people_low_risk + people_res_pos_given_low_risk_ag
            )

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag, size=people_symp_less_5d)
            )

            number_pcr_tests = people_symp_more_5d + people_res_neg_given_high_risk_ag

            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            people_confirmed_retest_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag**2, size=people_low_risk)
            )

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
                + people_confirmed_retest_ag
            )

            number_test_per_person = (number_pcr_tests + number_alt_tests) / (people)
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 4:
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d
            prob_model_first_responder_pcr = 0.01
            people_first_responder = np.sum(
                bernoulli.rvs(prob_model_first_responder_pcr, size=people)
            )
            people_high_risk = np.sum(
                bernoulli.rvs(
                    prob_model_high_risk, size=people - people_first_responder
                )
            )
            people_low_risk = people - people_first_responder - people_high_risk

            # Number of test
            number_pcr_tests = people_first_responder

            number_lamp_tests = (
                np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_lamp, size=people_high_risk
                    )
                )
                + people_high_risk
            )

            number_ag_tests = (
                np.sum(
                    bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
                )
                + people_low_risk
            )

            number_alt_tests = number_lamp_tests + number_ag_tests

            Highrisk = np.sum(
                bernoulli.rvs(prob_res_pos_given_high_risk_pcr, size=people_high_risk)
            )

            number_positive_reported = (
                np.sum(bernoulli.rvs(prob_res_pos_pcr, size=people_first_responder))
                + Highrisk
                + np.sum(
                    bernoulli.rvs(1 - prob_res_pos_given_high_risk_pcr, size=Highrisk)
                )
                + np.sum(
                    bernoulli.rvs(
                        prob_res_pos_given_low_risk_ag**2, size=people_low_risk
                    )
                )
            )

            number_test_per_person = (number_pcr_tests + number_alt_tests) / (people)

            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost_alt = (
                cost_lamp * number_lamp_tests + cost_antigen * number_ag_tests
            )
            total_cost = total_cost_alt + total_cost_pcr

            stock_capacity = ((number_pcr_tests + number_alt_tests) / total_cost) * (
                (people_high_risk + people_low_risk + people_first_responder) / people
            )

            efficiency = (number_positive_reported / total_cost) * (
                (people_high_risk + people_low_risk + people_first_responder) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [
                        people_high_risk + people_low_risk + people_first_responder
                    ],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "people_first_responder": [people_first_responder],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "sens_lamp_test": [sens_lamp_test],
                    "spec_lamp_test": [spec_lamp_test],
                    "prob_model_first_responder_pcr,": [prob_model_first_responder_pcr],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )
            df_out = pd.concat([df_out, df2])

    return df_out

In [5]:
def r_covid_testing_strategies(
    prevalence,
    sens_model,
    spec_model,
    strategy,
    people,
    n_simul=1000,
    sens_ag_test=0.8,
    spec_ag_test=0.95,
    sens_pcr_test=0.95,
    spec_pcr_test=0.95,
    sens_lamp_test=0.95,
    spec_lamp_test=0.95,
    cost_pcr=100,
    cost_antigen=50,
    cost_lamp=20,
    group_size=5,
    budget=100000,
):
    resultados = pd.DataFrame()
    for s in range(n_simul):
        df = covid_testing_strategies(
            prevalence,
            sens_model,
            spec_model,
            sens_ag_test,
            spec_ag_test,
            sens_pcr_test,
            spec_pcr_test,
            sens_lamp_test,
            spec_lamp_test,
            people,
            cost_pcr,
            cost_antigen,
            cost_lamp,
            group_size,
            strategy,
            budget,
        )
        df["nsimul"] = s + 1
        resultados = pd.concat([resultados, df], ignore_index=True)
    return resultados

In [6]:
model_parameters = pd.read_pickle(filepath_or_buffer="data/model_parameters.pkl")

In [7]:
df_strategy_1 = r_covid_testing_strategies(
    strategy=1,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_1.to_pickle(path="data/df_strategy_1.pkl")
df_strategy_1.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,1,0.269096,1000,408,592,0.805556,0.764505,0.8,0.95,0.75,...,233,320,269.096355,1.370098,16300,23300,39600,1411.616162,808.080808,1
1,1,0.269096,1000,408,592,0.805556,0.764505,0.8,0.95,0.50,...,305,292,269.096355,1.227941,9800,30500,40300,1243.176179,724.565757,1
2,1,0.269096,1000,408,592,0.805556,0.764505,0.8,0.95,0.25,...,349,267,269.096355,1.107843,5150,34900,40050,1128.589263,666.666667,1
3,1,0.269096,1000,401,599,0.805556,0.764505,0.8,0.95,0.75,...,233,299,269.096355,1.324190,14900,23300,38200,1390.052356,782.722513,2
4,1,0.269096,1000,401,599,0.805556,0.764505,0.8,0.95,0.50,...,283,298,269.096355,1.189526,9700,28300,38000,1255.263158,784.210526,2
5,1,0.269096,1000,401,599,0.805556,0.764505,0.8,0.95,0.25,...,342,271,269.096355,1.099751,4950,34200,39150,1126.436782,692.209451,2
6,1,0.269096,1000,398,602,0.805556,0.764505,0.8,0.95,0.75,...,238,295,269.096355,1.301508,14000,23800,37800,1370.370370,780.423280,3
7,1,0.269096,1000,398,602,0.805556,0.764505,0.8,0.95,0.50,...,291,292,269.096355,1.208543,9500,29100,38600,1246.113990,756.476684,3
8,1,0.269096,1000,398,602,0.805556,0.764505,0.8,0.95,0.25,...,355,263,269.096355,1.103015,4200,35500,39700,1105.793451,662.468514,3
9,1,0.269096,1000,364,636,0.805556,0.764505,0.8,0.95,0.75,...,220,282,269.096355,1.370879,13950,22000,35950,1388.038943,784.422809,4


In [8]:
df_strategy_2 = r_covid_testing_strategies(
    strategy=2,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_2.to_pickle(path="data/df_strategy_2.pkl")
df_strategy_2.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,2,0.269096,1000,369,631,0.805556,0.764505,0.8,0.95,0.75,...,387.271754,323.271754,269.096355,0.656272,13450,38727.175405,52177.175405,1257.775548,619.565455,1
1,2,0.269096,1000,369,631,0.805556,0.764505,0.8,0.95,0.50,...,435.024862,287.024862,269.096355,0.598025,8150,43502.486177,51652.486177,1157.785241,555.684504,1
2,2,0.269096,1000,369,631,0.805556,0.764505,0.8,0.95,0.25,...,477.650380,271.650380,269.096355,0.566650,4450,47765.037973,52215.037973,1085.224490,520.253150,1
3,2,0.269096,1000,383,617,0.805556,0.764505,0.8,0.95,0.75,...,358.897272,321.897272,269.096355,0.666897,15400,35889.727200,51289.727200,1300.255058,627.605740,2
4,2,0.269096,1000,383,617,0.805556,0.764505,0.8,0.95,0.50,...,433.773826,311.773826,269.096355,0.630774,9850,43377.382587,53227.382587,1185.055126,585.739540,2
5,2,0.269096,1000,383,617,0.805556,0.764505,0.8,0.95,0.25,...,486.271754,277.271754,269.096355,0.573272,4350,48627.175405,52977.175405,1082.110833,523.379648,2
6,2,0.269096,1000,397,603,0.805556,0.764505,0.8,0.95,0.75,...,374.769682,331.769682,269.096355,0.664770,14500,37476.968224,51976.968224,1278.969715,638.301335,3
7,2,0.269096,1000,397,603,0.805556,0.764505,0.8,0.95,0.50,...,447.897272,302.897272,269.096355,0.646897,9950,44789.727200,54739.727200,1181.769265,553.340850,3
8,2,0.269096,1000,397,603,0.805556,0.764505,0.8,0.95,0.25,...,494.024862,301.024862,269.096355,0.603025,5450,49402.486177,54852.486177,1099.357393,548.789823,3
9,2,0.269096,1000,388,612,0.805556,0.764505,0.8,0.95,0.75,...,384.893128,325.893128,269.096355,0.674893,14500,38489.312838,52989.312838,1273.640084,615.016710,4


In [9]:
df_strategy_3 = r_covid_testing_strategies(
    strategy=3,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_3.to_pickle(path="data/df_strategy_3.pkl")
df_strategy_3.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,3,0.269096,1000,399,601,0.805556,0.764505,0.8,0.95,0.75,...,232,307,269.096355,1.197,48250,23200,71450,1675.297411,429.671099,1
1,3,0.269096,1000,399,601,0.805556,0.764505,0.8,0.95,0.50,...,275,303,269.096355,1.120,42250,27500,69750,1605.734767,434.408602,1
2,3,0.269096,1000,399,601,0.805556,0.764505,0.8,0.95,0.25,...,347,278,269.096355,1.116,38450,34700,73150,1525.632262,380.041012,1
3,3,0.269096,1000,386,614,0.805556,0.764505,0.8,0.95,0.75,...,222,305,269.096355,1.179,47850,22200,70050,1683.083512,435.403283,2
4,3,0.269096,1000,386,614,0.805556,0.764505,0.8,0.95,0.50,...,289,281,269.096355,1.144,42750,28900,71650,1596.650384,392.184229,2
5,3,0.269096,1000,386,614,0.805556,0.764505,0.8,0.95,0.25,...,342,261,269.096355,1.100,37900,34200,72100,1525.658807,361.997226,2
6,3,0.269096,1000,409,591,0.805556,0.764505,0.8,0.95,0.75,...,250,316,269.096355,1.203,47650,25000,72650,1655.884377,434.962147,3
7,3,0.269096,1000,409,591,0.805556,0.764505,0.8,0.95,0.50,...,296,297,269.096355,1.150,42700,29600,72300,1590.594744,410.788382,3
8,3,0.269096,1000,409,591,0.805556,0.764505,0.8,0.95,0.25,...,360,271,269.096355,1.078,35900,36000,71900,1499.304590,376.912378,3
9,3,0.269096,1000,398,602,0.805556,0.764505,0.8,0.95,0.75,...,231,307,269.096355,1.182,47550,23100,70650,1673.036093,434.536447,4


In [10]:
df_strategy_4 = r_covid_testing_strategies(
    strategy=4,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_4.to_pickle(path="data/df_strategy_4.pkl")
df_strategy_4.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,people_first_responder,sens_model,spec_model,sens_ag_test,spec_ag_test,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,4,0.269096,1000,376,615,9,0.805556,0.764505,0.8,0.95,...,9,305,269.096355,1.222,44210,900,45110,2708.933718,676.125028,1
1,4,0.269096,1000,368,623,9,0.805556,0.764505,0.8,0.95,...,9,311,269.096355,1.207,44330,900,45230,2668.582799,687.596728,1
2,4,0.269096,1000,384,603,13,0.805556,0.764505,0.8,0.95,...,13,307,269.096355,1.176,42820,1300,44120,2665.457842,695.829556,1
3,4,0.269096,1000,396,595,9,0.805556,0.764505,0.8,0.95,...,9,347,269.096355,1.200,43050,900,43950,2730.375427,789.533561,2
4,4,0.269096,1000,391,596,13,0.805556,0.764505,0.8,0.95,...,13,307,269.096355,1.211,43460,1300,44760,2705.540661,685.880250,2
5,4,0.269096,1000,395,592,13,0.805556,0.764505,0.8,0.95,...,13,324,269.096355,1.190,42530,1300,43830,2715.035364,739.219713,2
6,4,0.269096,1000,403,588,9,0.805556,0.764505,0.8,0.95,...,9,368,269.096355,1.212,43320,900,44220,2740.841248,832.202623,3
7,4,0.269096,1000,384,604,12,0.805556,0.764505,0.8,0.95,...,12,321,269.096355,1.205,43270,1200,44470,2709.691927,721.834945,3
8,4,0.269096,1000,380,604,16,0.805556,0.764505,0.8,0.95,...,16,336,269.096355,1.210,43620,1600,45220,2675.807165,743.034056,3
9,4,0.269096,1000,366,626,8,0.805556,0.764505,0.8,0.95,...,8,293,269.096355,1.171,43420,800,44220,2648.123021,662.596110,4
